Midsemester Group 8

Members:
Kirk Kudoto
Welile N. Dlamini

21st July 2024



In [ ]:
!pip install timm

In [ ]:
import timm

In [ ]:
import urllib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Splitting a video into frames

In [ ]:
import cv2
import os
import shutil

def split_video_to_frames(video_path, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
        os.makedirs(output_folder)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    frame_count = 0

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # If the frame was not retrieved, then we have reached the end of the video
        if not ret:
            break

        # Save the frame as an image
        frame_filename = os.path.join(output_folder, f"frame_{frame_count:05d}.jpg")
        cv2.imwrite(frame_filename, frame)

        frame_count += 1

    # When everything done, release the video capture object
    cap.release()
    print(f"Extracted {frame_count} frames to {output_folder}")

In [ ]:
find = input("What are you looking for?")

What are you looking for?soccer ball


In [ ]:
def finder(list, x):
  count = 0

  for i in list:
    if x in i:
      return count
    else:
      count += 1

Trying work on individual frames

In [ ]:
def get_predictions(image_path):
  img = Image.open(image_path)

  from prettytable import PrettyTable
  table = PrettyTable(["Categories", "Probability"])

  tensor = transform(img).unsqueeze(0)

  with torch.no_grad():
    out = model(tensor)
  probabilities = torch.nn.functional.softmax(out[0], dim=0)
  print(probabilities.shape)

  # Get imagenet class mappings
  url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
  urllib.request.urlretrieve(url, filename)

  with open("imagenet_classes.txt", "r") as f:
      categories = [s.strip() for s in f.readlines()]

  # Print top categories per image
  top5_prob, top5_catid = torch.topk(probabilities, 5)
  for i in range(top5_prob.size(0)):
      table.add_row([categories[top5_catid[i]], str(round(top5_prob[i].item()*100, 2))+"%"])

Allowing the user to search

In [ ]:
import os

In [ ]:
# Function to search for an object in frames
def search_object_in_frames(folder, search_object):
    object_found = False
    for filename in sorted(os.listdir(folder)):
        image_path = os.path.join(folder, filename)
        img = Image.open(image_path)
        tensor = transform(img).unsqueeze(0)

        with torch.no_grad():
            out = model(tensor)
        probabilities = torch.nn.functional.softmax(out[0], dim=0)

        # Find the index of the search object in categories
        try:
            object_idx = categories.index(search_object)
        except ValueError:
            return f"Error: '{search_object}' is not a recognizable object in ImageNet classes."

        # Get the probability of the object being in the frame
        object_probability = probabilities[object_idx]

        # Check if the probability is above a threshold
        if object_probability.item() > 0.9:
            print(f"Object found in {filename} with probability {object_probability.item() * 100:.2f}%")
            object_found = True
            img.show()

    if not object_found:
        print("Object doesn't exist!!!")

In [ ]:
def get_highest_prob(file_probs):
  highest_prob = file_probs[0][-1]
  highest_file = file_probs[0][0]
  highest_index = 0

  for ind, i in enumerate(file_probs):
    if i[-1] > highest_prob:
      highest_prob = i[-1]
      highest_file = i[0]
      highest_index = ind
  return highest_index

In [ ]:
# Function to search for an object in frames
def search_object_in_frames(folder, search_object):
    searches = list()
    object_found = False
    for filename in sorted(os.listdir(folder)):
        image_path = os.path.join(folder, filename)
        img = Image.open(image_path)
        tensor = transform(img).unsqueeze(0)

        with torch.no_grad():
            out = model(tensor)
        probabilities = torch.nn.functional.softmax(out[0], dim=0)

        # Find the index of the search object in categories
        try:
            object_idx = categories.index(search_object)
        except ValueError:
            return f"Error: '{search_object}' is not a recognizable object in ImageNet classes."

        # Get the probability of the object being in the frame
        object_probability = probabilities[object_idx]

        # Check if the probability is above a threshold
        if object_probability.item() > 0.1:
            next = [filename, object_probability.item() * 100]
            searches.append(next)
            object_found = True

    for i in searches:
      img = Image.open(os.path.join(folder, i[0]))
      img.show()

    if not object_found:
        print("Object doesn't exist!!!")

In [ ]:
!pip install pyngrok==7.2.0

In [ ]:
!pip install streamlit

In [ ]:
%%writefile app.py
import streamlit as st
import torch
from PIL import Image
from torchvision import transforms
import timm
import pandas as pd

def load_model():
    """Load and return the pretrained Inception V3 model."""
    model = timm.create_model('inception_v3', pretrained=True)
    model.eval()
    return model

def transform_image(image):
    """Apply transformations to the image and return a tensor."""
    from timm.data import resolve_data_config
    from timm.data.transforms_factory import create_transform
    config = resolve_data_config({}, model = model)
    transform = create_transform(**config)
    return transform(image).unsqueeze(0)

def predict(image, model):
    """Predict the class probabilities of the image with the model."""
    tensor = transform_image(image)
    with torch.no_grad():
        outputs = model(tensor)
    return torch.nn.functional.softmax(outputs[0], dim=0)

import cv2
import os
import shutil

def split_video_to_frames(video_path, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
        os.makedirs(output_folder)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    frame_count = 0

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # If the frame was not retrieved, then we have reached the end of the video
        if not ret:
            break

        # Save the frame as an image
        frame_filename = os.path.join(output_folder, f"frame_{frame_count:05d}.jpg")
        cv2.imwrite(frame_filename, frame)

        frame_count += 1

    # When everything done, release the video capture object
    cap.release()
    print(f"Extracted {frame_count} frames to {output_folder}")

def search_object_in_frames(folder, search_object):
    object_found = False
    for filename in sorted(os.listdir(folder)):
        image_path = os.path.join(folder, filename)
        img = Image.open(image_path)
        tensor = transform(img).unsqueeze(0)

        with torch.no_grad():
            out = model(tensor)
        probabilities = torch.nn.functional.softmax(out[0], dim=0)

        # Find the index of the search object in categories
        try:
            object_idx = categories.index(search_object)
        except ValueError:
            return f"Error: '{search_object}' is not a recognizable object in ImageNet classes."

        # Get the probability of the object being in the frame
        object_probability = probabilities[object_idx]

        # Check if the probability is above a threshold
        if object_probability.item() > 0.9:
            print(f"Object found in {filename} with probability {object_probability.item() * 100:.2f}%")
            object_found = True
            img.show()

    if not object_found:
        print("Object doesn't exist!!!")

def main():
    st.title("Image Classifier Using InceptionV3")
    uploaded_file = st.file_uploader("Upload a video...", type=["mov", "avi", "mp4"])

    search_element = st.text_input("What are you looking for?")

    if uploaded_file is not None:
        split_video_to_frames(uploaded_file, '/content/drive/MyDrive/GoogleInceptionModel/current_frames')
        search_object_in_frames('/content/drive/MyDrive/GoogleInceptionModel/current_frames', search_element)

if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:
!export STREAMLIT_SERVER_MAX_UPLOAD_SIZE=40

In [ ]:
import streamlit as st

def check_config():
    st.write("Current max upload size:", st.config.get_option("server.maxUploadSize"))


check_config()

In [ ]:
%%writefile app.py
import streamlit as st
import torch
from PIL import Image
import timm
from torchvision import transforms
import cv2
import os
import shutil
import time
import urllib

MAX_FILE_SIZE = 40

def load_model():
    """Load and return the pretrained Inception V3 model."""
    model = timm.create_model('inception_v3', pretrained=True)
    model.eval()
    return model

url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
urllib.request.urlretrieve(url, filename)

with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]


def transform_image(image, model):
    """Apply transformations to the image and return a tensor."""
    from timm.data import resolve_data_config
    from timm.data.transforms_factory import create_transform
    config = resolve_data_config({}, model=model)
    transform = create_transform(**config)
    return transform(image).unsqueeze(0)

def split_video_to_frames(video_file, output_folder):
    # Save the uploaded video file to a local path
    video_path = '/tmp/temp_video.mp4'
    with open(video_path, 'wb') as file:
        file.write(video_file.read())

    # Create the output folder if it doesn't exist
    if os.path.exists(output_folder):
        try:
            shutil.rmtree(output_folder)
            time.sleep(1)  # Pause to ensure the deletion completes
        except OSError as e:
            print(f"Error: Could not completely clear the folder {output_folder}: {e}")


    # Ensure the folder is created
    os.makedirs(output_folder, exist_ok=True)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    frame_count = 0

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # If the frame was not retrieved, then we have reached the end of the video
        if not ret:
            break

        # Save the frame as an image
        frame_filename = os.path.join(output_folder, f"frame_{frame_count:05d}.jpg")
        cv2.imwrite(frame_filename, frame)

        frame_count += 1

    # When everything done, release the video capture object
    cap.release()
    print(f"Extracted {frame_count} frames to {output_folder}")

import streamlit as st
import tempfile
import shutil

def main():

    st.title("Video Analysis and Object Finding Using Inception V3")
    st.write("Current max upload size:", st.config.get_option("server.maxUploadSize"))
    uploaded_file = st.file_uploader("Upload a video...", type=["mov", "avi", "mp4"])
    search_element = st.text_input("What are you looking for?").lower()

    if uploaded_file is not None:
        # Notify the user that the upload was successful and processing will start
        st.success("Upload successful! Processing the video...")

        # Create a temporary directory to store the video frames
        with tempfile.TemporaryDirectory() as tmpdirname:
            # Handle the video upload and frame splitting
            try:
                split_video_to_frames(uploaded_file, tmpdirname)
                st.success("Video has been split into frames.")
            except Exception as e:
                st.error(f"Failed to split video into frames: {e}")

            # Perform object search if a search term is provided
            if search_element:
                try:
                    result = search_object_in_frames(tmpdirname, search_element)
                    if result:
                        st.success(f"Found '{search_element}' in the video.")
                    else:
                        st.warning(f"Could not find '{search_element}' in the video.")
                except Exception as e:
                    st.error(f"Failed to search for object: {e}")
            else:
                st.warning("Please enter an object to search for in the video frames.")

        if st.button("Click to show instructions"):
            st.write("""
                Please upload a video file (MOV, AVI, MP4) and enter an object name you wish to search for.
                The system will process the video, split it into frames, and look for the specified object across the frames.
            """)

def split_video_to_frames(video_file, output_folder):
    # Write the uploaded video file to a temporary file
    video_path = tempfile.NamedTemporaryFile(delete=False, suffix='.mp4').name
    with open(video_path, 'wb') as file:
        file.write(video_file.read())

    # Ensure the output directory is clean
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
    os.makedirs(output_folder, exist_ok=True)

    # Process the video file
    import cv2
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        st.error("Failed to open video file.")
        return

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_filename = os.path.join(output_folder, f"frame_{frame_count:05d}.jpg")
        cv2.imwrite(frame_filename, frame)
        frame_count += 1

    cap.release()
    st.info(f"Extracted {frame_count} frames to {output_folder}")

def search_object_in_frames(folder, search_object):
    from PIL import Image
    import torch

    model = load_model()  # Make sure to define load_model or import it if it's not in this script
    object_found = False
    for filename in sorted(os.listdir(folder)):
        image_path = os.path.join(folder, filename)
        img = Image.open(image_path)
        tensor = transform_image(img, model)  # Ensure transform_image is defined or imported

        with torch.no_grad():
            outputs = model(tensor)
        probabilities = torch.nn.functional.softmax(outputs[0], dim=0)

        try:
            object_idx = categories.index(search_object)  # 'categories' should be defined elsewhere
        except ValueError:
            st.error(f"'{search_object}' is not a recognizable object in ImageNet classes.")
            return False

        object_probability = probabilities[object_idx]
        if object_probability.item() > 0.3:  # Adjusting the threshold to 0.3 as necessary
            object_found = True
            st.image(img, channels = "BGR", caption = f"Object found in {filename} with probability {object_probability.item() * 100:.2f}%")

    if not object_found:
        st.warning("Object does not exist!!!")
    return object_found

if __name__ == "__main__":
    main()



Overwriting app.py


In [ ]:
import os
import signal

# List all running processes and find the ngrok process
!ps -aux | grep ngrok


# Assuming the ngrok process ID (PID) is at index 1 of the output
output = !ps -aux | grep ngrok
for line in output:
    if 'ngrok' in line and 'grep' not in line:
        pid = int(line.split()[1])
        # Kill the ngrok process
        os.kill(pid, signal.SIGTERM)
        print(f"ngrok process with PID {pid} has been terminated.")


root        5498  0.0  0.0   7376  3416 ?        S    23:35   0:00 /bin/bash -c ps -aux | grep ngrok
root        5500  0.0  0.0   7376   280 ?        R    23:35   0:00 /bin/bash -c ps -aux | grep ngrok


In [ ]:
import subprocess
import os
from pyngrok import ngrok
import signal

# Function to kill any existing ngrok processes
def kill_ngrok():
    ngrok.kill()

# Set the ngrok auth token
ngrok.set_auth_token("2jZqPXDqGHUARxWpfOS6ZymJiY8_4Udqg3peujBz4rSheHLMZ")

# Kill existing ngrok processes
kill_ngrok()

# Start the Streamlit app in a subprocess
process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Connect to the ngrok tunnel on a potentially new port
try:
    public_url = ngrok.connect(addr='8501', proto='http')
    print(f'Streamlit at {public_url}')
except Exception as e:
    print("Failed to create ngrok tunnel:", e)
    kill_ngrok()
    # Attempt to reconnect
    public_url = ngrok.connect(addr='8501', proto='http')
    print(f'Streamlit at {public_url}')


INFO:pyngrok.process:Updating authtoken for default "config_path" of "ngrok_path": /root/.config/ngrok/ngrok
2024-07-21 23:36:11.355 Updating authtoken for default "config_path" of "ngrok_path": /root/.config/ngrok/ngrok
INFO:pyngrok.ngrok:Opening tunnel named: http-8501-21620506-58d7-4ebe-be53-c1846529ae7d
2024-07-21 23:36:11.419 Opening tunnel named: http-8501-21620506-58d7-4ebe-be53-c1846529ae7d
INFO:pyngrok.process.ngrok:t=2024-07-21T23:36:11+0000 lvl=info msg="no configuration paths supplied"
2024-07-21 23:36:11.493 t=2024-07-21T23:36:11+0000 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2024-07-21T23:36:11+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
2024-07-21 23:36:11.497 t=2024-07-21T23:36:11+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
INFO:pyngrok.process.ngrok:t=2024-07-21T23:36:11+0000 lvl=info msg="open config file" path=/root/.config/ngr

Streamlit at NgrokTunnel: "https://d0f8-34-125-237-21.ngrok-free.app" -> "http://localhost:8501"


2024-07-21 23:36:11.930 t=2024-07-21T23:36:11+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8501-21620506-58d7-4ebe-be53-c1846529ae7d addr=http://localhost:8501 url=https://d0f8-34-125-237-21.ngrok-free.app
INFO:pyngrok.process.ngrok:t=2024-07-21T23:36:11+0000 lvl=info msg=end pg=/api/tunnels id=340f8ffd54b6da50 status=201 dur=71.517522ms
2024-07-21 23:36:11.933 t=2024-07-21T23:36:11+0000 lvl=info msg=end pg=/api/tunnels id=340f8ffd54b6da50 status=201 dur=71.517522ms
